Matthew Peltier 7.2 homework

In [1]:
%matplotlib inline
from pathlib import Path
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
from dmba import regressionSummary, classificationSummary, liftChart, gainsChart
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
import matplotlib.pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
# Working directory:
#
# We assume that data are kept in the same directory as the notebook. If you keep your 
# data in a different folder, replace the argument of the `Path`
DATA = Path('/Users/mgpel/Downloads/')
#DATA = Path('/Users/Agent/Downloads/dmba')
#DATA = Path('C:/Users/user/dmba')
# and then load data using 
#
df = pd.read_csv(DATA / 'UniversalBank.csv')


In [4]:
#cleaned dataframe without zip and ID
df.drop(['ID', 'ZIP Code'], axis=1, inplace=True)
# Display the updated DataFrame

In [5]:
df

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,3,0,0,0,0,1,0
4996,30,4,15,4,0.4,1,85,0,0,0,1,0
4997,63,39,24,2,0.3,3,0,0,0,0,0,0
4998,65,40,49,3,0.5,2,0,0,0,0,1,0


In [6]:
# Create dummy variables for the Education column
education_dummies = pd.get_dummies(df['Education'], prefix='Education', drop_first=False)

# Concatenate the original DataFrame with the dummy variables
df = pd.concat([df, education_dummies], axis=1)

# Drop the original Education column if needed
df.drop('Education', axis=1, inplace=True)

# Display the updated DataFrame
df

,Age,Experience,Income,Family,CCAvg,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,Education_1,Education_2,Education_3
0,25,1,49,4,1.6,0,0,1,0,0,0,1,0,0
1,45,19,34,3,1.5,0,0,1,0,0,0,1,0,0
2,39,15,11,1,1.0,0,0,0,0,0,0,1,0,0
3,35,9,100,1,2.7,0,0,0,0,0,0,0,1,0
4,35,8,45,4,1.0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,0,0,0,0,1,0,0,0,1
4996,30,4,15,4,0.4,85,0,0,0,1,0,1,0,0
4997,63,39,24,2,0.3,0,0,0,0,0,0,0,0,1
4998,65,40,49,3,0.5,0,0,0,0,1,0,0,1,0


In [7]:
#dataframe split into training %60 and validation %40 sets
trainData, validData = train_test_split(df, test_size = .4, random_state = 1)
print(trainData.shape, validData.shape)

(3000, 14) (2000, 14)


In [8]:
#new customer used to compare data
new_customer = pd.DataFrame([{'Age': 40, 'Experience' : 10, 'Income': 84, 'Family' : 2, 'CCAvg': 2, 'Education_1': 0, 'Education_2': 1, 'Education_3': 0, 'Mortgage': 0, 'Securities Account': 0, 'CD Account': 0, 'Online': 1, 'Credit Card': 1}])
new_customer

,Age,Experience,Income,Family,CCAvg,Education_1,Education_2,Education_3,Mortgage,Securities Account,CD Account,Online,Credit Card
0,40,10,84,2,2,0,1,0,0,0,0,1,1


In [9]:
#training data 
trainData

,Age,Experience,Income,Family,CCAvg,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,Education_1,Education_2,Education_3
4522,31,5,29,1,0.3,0,0,0,0,0,0,1,0,0
2851,61,36,81,3,1.8,0,0,0,0,1,0,0,1,0
2313,58,32,54,3,0.3,0,0,0,0,1,1,0,1,0
982,58,33,52,3,0.5,0,0,0,0,0,1,0,1,0
1164,41,17,94,3,3.8,327,1,1,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,60,36,39,4,1.3,140,0,0,0,1,0,0,1,0
2763,55,31,13,4,0.7,0,0,0,0,1,0,1,0,0
905,46,22,28,1,1.0,84,0,0,0,1,1,1,0,0
3980,46,22,89,4,1.4,0,0,0,0,1,0,0,1,0


In [10]:
#validation data
validData

,Age,Experience,Income,Family,CCAvg,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,Education_1,Education_2,Education_3
2764,31,5,84,1,2.9,105,0,0,0,0,1,0,0,1
4767,35,9,45,3,0.9,101,0,1,0,0,0,1,0,0
3814,34,9,35,3,1.3,0,0,0,0,0,0,1,0,0
3499,49,23,114,1,0.3,286,0,0,0,1,0,1,0,0
2735,36,12,70,3,2.6,165,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4372,34,10,41,1,2.4,0,0,0,0,0,0,0,1,0
3401,39,15,28,1,1.4,118,0,0,0,1,1,0,0,1
1239,51,26,12,2,0.7,109,0,0,0,1,1,0,0,1
437,36,9,31,4,1.0,0,0,1,0,1,0,0,1,0


In [11]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Normalize the DataFrame using the fitted scaler
scaler = preprocessing.StandardScaler()
scaler.fit(trainData[['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Securities Account', 'CD Account', 'Online', 'CreditCard', 'Education_1', 'Education_2', 'Education_3']])

norm_df = pd.concat([pd.DataFrame(scaler.transform(df[['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Securities Account', 'CD Account', 'Online', 'CreditCard', 'Education_1', 'Education_2', 'Education_3']]), 
                                        columns = ['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zMortgage', 'zSecurities Account', 'zCD Account', 'zOnline', 'zCreditCard', 'zEducation_1', 'zEducation_2', 'zEducation_3']),
                               df[['Personal Loan']]], axis = 1)#success class  as 1loan acceptance
trainNorm = norm_df.iloc[trainData.index]
validNorm = norm_df.iloc[validData.index]
new_customer = pd.DataFrame(scaler.transform(new_customer),
        columns = ['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zMortgage', 'zSecurities Account', 'zCD Account', 'zOnline', 'zCreditCard', 'zEducation_1', 'zEducation_2', 'zEducation_3'])
new_Norm = pd.DataFrame(scaler.transform(new_customer), columns=['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zMortgage', 'zSecurities Account', 'zCD Account', 'zOnline', 'zCreditCard', 'zEducation_1', 'zEducation_2', 'zEducation_3'])

In [12]:
#normalized data frame
norm_df

,zAge,zExperience,zIncome,zFamily,zCCAvg,zMortgage,zSecurities Account,zCD Account,zOnline,zCreditCard,zEducation_1,zEducation_2,zEducation_3,Personal Loan
0,-1.793861,-1.685278,-0.534478,1.393956,-0.193995,-0.559242,2.967140,-0.252646,-1.198767,-0.65050,1.192186,-0.628252,-0.660895,0
1,-0.050641,-0.116848,-0.858208,0.520915,-0.251416,-0.559242,2.967140,-0.252646,-1.198767,-0.65050,1.192186,-0.628252,-0.660895,0
2,-0.573607,-0.465388,-1.354594,-1.225168,-0.538521,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,1.192186,-0.628252,-0.660895,0
3,-0.922251,-0.988198,0.566204,-1.225168,0.437636,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,-0.838795,1.591719,-0.660895,0
4,-0.922251,-1.075333,-0.620806,1.393956,-0.538521,-0.559242,-0.337025,-0.252646,-1.198767,1.53728,-0.838795,1.591719,-0.660895,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-1.445217,-1.511008,-0.728716,-1.225168,-0.021732,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,-0.838795,-0.628252,1.513101,0
4996,-1.358056,-1.423873,-1.268266,1.393956,-0.883047,0.282120,-0.337025,-0.252646,0.834190,-0.65050,1.192186,-0.628252,-0.660895,0
4997,1.518258,1.625852,-1.074028,-0.352127,-0.940468,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,-0.838795,-0.628252,1.513101,0
4998,1.692580,1.712987,-0.534478,0.520915,-0.825626,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,-0.838795,1.591719,-0.660895,0


In [13]:
#Normalized training data
trainNorm

,zAge,zExperience,zIncome,zFamily,zCCAvg,zMortgage,zSecurities Account,zCD Account,zOnline,zCreditCard,zEducation_1,zEducation_2,zEducation_3,Personal Loan
4522,-1.270895,-1.336738,-0.966118,-1.225168,-0.940468,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,1.192186,-0.628252,-0.660895,0
2851,1.343936,1.364447,0.156146,0.520915,-0.079153,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,-0.838795,1.591719,-0.660895,0
2313,1.082453,1.015907,-0.426568,0.520915,-0.940468,-0.559242,-0.337025,-0.252646,0.834190,1.53728,-0.838795,1.591719,-0.660895,0
982,1.082453,1.103042,-0.469732,0.520915,-0.825626,-0.559242,-0.337025,-0.252646,-1.198767,1.53728,-0.838795,1.591719,-0.660895,0
1164,-0.399285,-0.291118,0.436712,0.520915,1.069267,2.677526,2.967140,3.958114,0.834190,-0.65050,-0.838795,1.591719,-0.660895,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,1.256775,1.364447,-0.750298,1.393956,-0.366258,0.826531,-0.337025,-0.252646,0.834190,-0.65050,-0.838795,1.591719,-0.660895,0
2763,0.820970,0.928772,-1.311430,1.393956,-0.710784,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,1.192186,-0.628252,-0.660895,0
905,0.036520,0.144557,-0.987700,-1.225168,-0.538521,0.272222,-0.337025,-0.252646,0.834190,1.53728,1.192186,-0.628252,-0.660895,0
3980,0.036520,0.144557,0.328802,1.393956,-0.308837,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,-0.838795,1.591719,-0.660895,0


In [14]:
#normalized training data
validNorm

,zAge,zExperience,zIncome,zFamily,zCCAvg,zMortgage,zSecurities Account,zCD Account,zOnline,zCreditCard,zEducation_1,zEducation_2,zEducation_3,Personal Loan
2764,-1.270895,-1.336738,0.220892,-1.225168,0.552478,0.480088,-0.337025,-0.252646,-1.198767,1.53728,-0.838795,-0.628252,1.513101,0
4767,-0.922251,-0.988198,-0.620806,0.520915,-0.595942,0.440494,2.967140,-0.252646,-1.198767,-0.65050,1.192186,-0.628252,-0.660895,0
3814,-1.009412,-0.988198,-0.836626,0.520915,-0.366258,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,1.192186,-0.628252,-0.660895,0
3499,0.298004,0.231692,0.868352,-1.225168,-0.940468,2.271693,-0.337025,-0.252646,0.834190,-0.65050,1.192186,-0.628252,-0.660895,0
2735,-0.835090,-0.726793,-0.081256,0.520915,0.380215,1.073990,-0.337025,-0.252646,0.834190,-0.65050,-0.838795,1.591719,-0.660895,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4372,-1.009412,-0.901063,-0.707134,-1.225168,0.265373,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,-0.838795,1.591719,-0.660895,0
3401,-0.573607,-0.465388,-0.987700,-1.225168,-0.308837,0.608766,-0.337025,-0.252646,0.834190,1.53728,-0.838795,-0.628252,1.513101,0
1239,0.472326,0.493097,-1.333012,-0.352127,-0.710784,0.519681,-0.337025,-0.252646,0.834190,1.53728,-0.838795,-0.628252,1.513101,0
437,-0.835090,-0.988198,-0.922954,1.393956,-0.538521,-0.559242,2.967140,-0.252646,0.834190,-0.65050,-0.838795,1.591719,-0.660895,0


In [15]:
new_Norm

,zAge,zExperience,zIncome,zFamily,zCCAvg,zMortgage,zSecurities Account,zCD Account,zOnline,zCreditCard,zEducation_1,zEducation_2,zEducation_3
0,-4.015286,-1.850928,-1.587229,-2.405631,-1.092238,-0.564777,9.466897,-1.316475,-3.63581,-2.07365,-2.542373,2.905317,2.628579


In [16]:

#use nearest neighbors from scikit learn to compute knn
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=3)
knn.fit(trainNorm[['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zMortgage', 'zSecurities Account', 'zCD Account', 'zOnline', 'zCreditCard', 'zEducation_1', 'zEducation_2', 'zEducation_3']])
distances, indices = knn.kneighbors(new_Norm)
#indices is a list of lists we are only interested in the first element
trainNorm.iloc[indices[0], :]




,zAge,zExperience,zIncome,zFamily,zCCAvg,zMortgage,zSecurities Account,zCD Account,zOnline,zCreditCard,zEducation_1,zEducation_2,zEducation_3,Personal Loan
3796,-1.881022,-1.946683,-0.512896,0.520915,0.265373,-0.559242,2.96714,-0.252646,-1.198767,-0.6505,-0.838795,1.591719,-0.660895,0
4044,-0.835090,-0.813928,-1.397758,-0.352127,-0.940468,0.420697,2.96714,-0.252646,-1.198767,-0.6505,-0.838795,1.591719,-0.660895,0
3070,-1.532378,-1.511008,0.005072,-0.352127,-0.079153,1.628299,2.96714,-0.252646,-1.198767,-0.6505,-0.838795,1.591719,-0.660895,0


In [17]:
indices

array([[2769, 1995, 1315]], dtype=int64)

In [18]:
trainNorm

,zAge,zExperience,zIncome,zFamily,zCCAvg,zMortgage,zSecurities Account,zCD Account,zOnline,zCreditCard,zEducation_1,zEducation_2,zEducation_3,Personal Loan
4522,-1.270895,-1.336738,-0.966118,-1.225168,-0.940468,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,1.192186,-0.628252,-0.660895,0
2851,1.343936,1.364447,0.156146,0.520915,-0.079153,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,-0.838795,1.591719,-0.660895,0
2313,1.082453,1.015907,-0.426568,0.520915,-0.940468,-0.559242,-0.337025,-0.252646,0.834190,1.53728,-0.838795,1.591719,-0.660895,0
982,1.082453,1.103042,-0.469732,0.520915,-0.825626,-0.559242,-0.337025,-0.252646,-1.198767,1.53728,-0.838795,1.591719,-0.660895,0
1164,-0.399285,-0.291118,0.436712,0.520915,1.069267,2.677526,2.967140,3.958114,0.834190,-0.65050,-0.838795,1.591719,-0.660895,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,1.256775,1.364447,-0.750298,1.393956,-0.366258,0.826531,-0.337025,-0.252646,0.834190,-0.65050,-0.838795,1.591719,-0.660895,0
2763,0.820970,0.928772,-1.311430,1.393956,-0.710784,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,1.192186,-0.628252,-0.660895,0
905,0.036520,0.144557,-0.987700,-1.225168,-0.538521,0.272222,-0.337025,-0.252646,0.834190,1.53728,1.192186,-0.628252,-0.660895,0
3980,0.036520,0.144557,0.328802,1.393956,-0.308837,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,-0.838795,1.591719,-0.660895,0


In [19]:
distances

array([[9.10591621, 9.1229078 , 9.255272  ]])

In [20]:
indices

array([[2769, 1995, 1315]], dtype=int64)

7.2.b, What is a choice of k that balances between overfitting and ignoring the predictor information

__7.2.c.__ Show the confusion matrix for the validation data that results from using the best k.

__7.2.d.__ Consider the following customer: Age = 40, Experience = 10, Income = 84, Family = 2, CCAvg = 2, Education_1 = 0, Education_2 = 1, Education_3 = 0, Mortgage = 0, Securities Account = 0, CD Account = 0, Online = 1 and Credit Card = 1. Classify the customer using the best k.

__7.2.e.__ Repartition the data, this time into training, validation, and test sets (50% : 30% : 20%). Apply the <i>k</i>-NN method with the k chosen above. Compare the confusion matrix of the test set with that of the training and validation sets. Comment on the differences and their reason.

__7.2.f.__ The hyperparameter k should be chosen using cross-validation. Review Chapters 15.2 and 15.3 of Deitel - Intro to Python first. Choose k based on 10-fold <b>cross-validation</b> in Chapter 2.5. Note you need to normalize data based on training set. Review the documentation at https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html for sklearn.model_selection.KFold, especially the split() method that generates indices to split data into training and test set and normalize data based on training set for cross-validation.

# Problem 7.3 Predicting Housing Median Prices

The file _BostonHousing.csv_ contains information on over 500 census tracts in Boston, where for each tract multiple variables
are recorded. The last column (CAT. MEDV) was derived from MEDV, such that it obtains the value 1 if MEDV > 30 and 0 otherwise. Consider the goal of predicting the median value (MEDV) of a tract, given the information in the first 12 columns. Review the data dictionary at https://docs.google.com/document/d/1BHznjIh1vBAA1sZq0q8-A5dd_CVf6MkFvT-VLq2h2UY/edit.
Partition the data into training (60%) and validation (40%) sets.

__7.3.a.__ Perform a <i>k</i>-NN prediction with all 12 predictors (ignore the CAT. MEDV column), trying values of <i>k</i> from 1 to 5. Make sure to normalize the data. What is the best <i>k</i>? What does it mean?

__7.3.b.__ Predict the MEDV for a tract with the following information, using the best k:

CRIM = 0.2, ZN = 0, INDUS = 7, CHAS = 0, NOX = 0.538, RM = 6, AGE = 62, DIS = 4.7, RAD = 4, TAX = 307, PTRATIO = 21, LSTAT = 10.

__7.3.c.__ If we used the above <i>k</i>-NN algorithm to score the training data, what would be the error of the training set?

__7.3.d.__ Why is the validation data error overly optimistic compared to the error rate when applying this <i>k</i>-NN predictor to new data?

__7.3.e.__ If the purpose is to predict MEDV for several thousands of new tracts, what would be the disadvantage of using <i>k</i>-NN prediction? List the operations that the algorithm goes through in order to produce each prediction.

__7.3.f.__  Choose k based on 10-fold cross-validation as in __7.2.f.__